# Expérimentations sur la centralité (visualisation 3)


On veut comparer la centralité des ports suivants :

```
{
    "main_port": "La Rochelle",
    "admiralties": ["Sables-d'Olonne", "La Rochelle", "Marennes"]
},
{
    "main_port": "Nantes",
    "admiralties": ["Nantes"]
},
{
    "main_port": "Bordeaux",
    "admiralties": ["Bordeaux"]
},
{
    "main_port": "Le Havre",
    "admiralties": ["Le Havre", "Havre"]
}
```


Méthodologie de constitution des réseaux et des métriques :

* je me fonde sur les flux navigo de 1787 (`get_flows`sur l'API)
* pour chaque région à comparer, je récupère tous les flux qui partent ou arrivent dans la région
* pour chacun des réseaux ainsi construits, je calcule la métrique de betweenness centrality (ou de pagerank) et je regarde celle du port principal

In [ ]:
from poitousprint import Portic
from vega import VegaLite
import pandas as pds

portic_client = Portic()

# pointcalls = portic_client.get_pointcalls(year = 1787)
flows = portic_client.get_flows(year = 1787)

# print('pointcalls : ', len(pointcalls), 'flows : ', len(flows))
print('flows : ', len(flows))

In [ ]:
ports_to_compare = [
    {
        "main_port": "La Rochelle",
        "admiralties": ["Sables-d'Olonne", "La Rochelle", "Marennes"]
    },
    {
        "main_port": "Nantes",
        "admiralties": ["Nantes"]
    },
    {
        "main_port": "Bordeaux",
        "admiralties": ["Bordeaux"]
    },
    {
        "main_port": "Le Havre",
        "admiralties": ["Le Havre", "Havre"]
    }
]

In [ ]:
import networkx as nx

def build_graph(name, flows, admiralties):
    graph = nx.DiGraph()

    def add_node(g, name, admiralty=None, peche=0):
        if admiralty is None:
            admiralty = 'n/a'
        g.add_node(
            name,
            admiralty=admiralty,
            internal= True if admiralty in admiralties else False,
            inside_degree=0,
            outside_degree=0
        )

    def add_edge(g, source, target, tonnage):
        if g.has_edge(source, target):
            attr = g[source][target]
            attr['weight'] += 1
            attr['tonnage'] += tonnage
        else:
            g.add_edge(
                source,
                target,
                weight=1,
                tonnage=tonnage
            )

    for flow in flows:
        
        source = flow['departure']
        target = flow['destination']

        source_admiralty = flow['departure_admiralty']
        target_admiralty = flow['destination_admiralty']

        concerns_flow = source_admiralty in admiralties or target_admiralty in admiralties
        
        if not concerns_flow:
            continue
            
        tonnage = 0

        try:
            tonnage = int(flow['tonnage'] if flow['tonnage'] is not None else 0)
        except ValueError:
            pass

        # Macro graph
        if source == target:
            add_node(graph, source, source_admiralty, 1)
        else:
            add_node(graph, source, source_admiralty)
            add_node(graph, target, target_admiralty)
            add_edge(graph, source, target, tonnage)

    
    nx.write_gexf(graph, './%s.gexf' % name)  
    return graph

In [ ]:
metrics = []

page_ranks = []
betweenness_centralities = []

for p in ports_to_compare:
    port, admiralties = p.values()
    graph = build_graph(port, flows, admiralties)
    page_rank = nx.pagerank(graph)
    betweenness_centrality =  nx.betweenness_centrality(graph)
    metrics.append({
        "port": port,
        "page_rank": page_rank[port],
        "betweenness_centrality": betweenness_centrality[port]
    })
    page_ranks += [{"group": port, "port": p, "page_rank": value} for (p, value) in page_rank.items() if graph.nodes[p]['internal'] == True]
    betweenness_centralities += [{"group": port, "port": p, "betweenness_centrality": value} for (p, value) in betweenness_centrality.items()if graph.nodes[p]['internal'] == True]

In [ ]:
from csv import DictWriter


with open('centrality_comparison.csv', 'w', newline='') as csvfile:
    fieldnames = metrics[0].keys()
    writer = DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for m in metrics:
        writer.writerow(m)
    

## Vue d'ensemble

In [ ]:
import pandas as pds

VegaLite({
    "title": "Distribution des pageranks",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "page_rank",
            "title": "pagerank de chaque port principal"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        }
    }
}, pds.DataFrame(metrics))

In [ ]:
VegaLite({
    "title": "Distribution des betweenness centralities",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "betweenness_centrality",
            "title": "pagerank de chaque port principal"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        }
    }
}, pds.DataFrame(metrics))

## Détails/contrôle des distributions et des effets des deux types de métriques

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des pageranks de La Rochelle (amirautés de Marennes, Sables d'Olonnes et La Rochelle)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "page_rank",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightgreen"]
            }
        }
    }
}, pds.DataFrame([port for port in page_ranks if port["group"] == 'La Rochelle']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des betweenness_centrality de La Rochelle (amirautés de Marennes, Sables d'Olonnes et La Rochelle)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "betweenness_centrality",
            "title": "betweenness_centrality de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightblue"]
            }
        }
    }
}, pds.DataFrame([port for port in betweenness_centralities if port["group"] == 'La Rochelle']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des pageranks de Nantes (amirauté de Nantes)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "page_rank",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightgreen"]
            }
        }
    }
}, pds.DataFrame([port for port in page_ranks if port["group"] == 'Nantes']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des betweenness_centrality de Nantes (amirauté de Nantes)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "betweenness_centrality",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightblue"]
            }
        }
    }
}, pds.DataFrame([port for port in betweenness_centralities if port["group"] == 'Nantes']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des pageranks de Bordeaux (amirauté de Bordeaux)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "page_rank",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightgreen"]
            }
        }
    }
}, pds.DataFrame([port for port in page_ranks if port["group"] == 'Bordeaux']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des betweenness_centrality de Bordeaux (amirauté de Bordeaux)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "betweenness_centrality",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightblue"]
            }
        }
    }
}, pds.DataFrame([port for port in betweenness_centralities if port["group"] == 'Bordeaux']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des pageranks du Havre (amirautés 'Havre' et 'Le Havre')",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "page_rank",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightgreen"]
            }
        }
    }
}, pds.DataFrame([port for port in page_ranks if port["group"] == 'Le Havre']))

In [ ]:
VegaLite({
    "title": "Comparaison de la distribution des betweenness_centrality du Havre (amirauté du Havre)",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "field": "betweenness_centrality",
            "title": "pagerank de chaque port"
        },
        "y": {
            "field": "port",
            "type": "nominal",

            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "group",
            "title": "région",
            "scale": {
                "range": ["lightblue"]
            }
        }
    }
}, pds.DataFrame([port for port in betweenness_centralities if port["group"] == 'Le Havre']))